In [3]:
pip install mahotas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import cv2
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
import os
import pandas as pd
from skimage.measure import shannon_entropy
import mahotas
from tqdm import tqdm
from scipy.stats import skew, kurtosis
import pywt

In [ ]:
def compute_zernike_moments(image, degree=8): # 25 features

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, new_img = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour = max(contours, key=cv2.contourArea)
    
    mask = np.zeros(new_img.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], contourIdx=-1, color=(255,), thickness=-1)
    binary = (mask > 0).astype(np.uint8)

    (x, y), radius = cv2.minEnclosingCircle(contour)
    x, y = int(x), int(y)
    radius = int(np.ceil(radius))
    x1 = max(x - radius, 0)
    y1 = max(y - radius, 0)
    x2 = x + radius
    y2 = y + radius

    cropped_mask = binary[y1:y2, x1:x2]

    h, w = cropped_mask.shape
    if h != w:
        size = max(h, w)
        square_mask = np.zeros((size, size), dtype=np.uint8)
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        square_mask[y_offset:y_offset+h, x_offset:x_offset+w] = cropped_mask
    else:
        square_mask = cropped_mask

    effective_radius = square_mask.shape[0] // 2
    zernike_moments = mahotas.features.zernike_moments(square_mask, effective_radius, degree)

    return {f"Zernike_{i}": zernike_moments[i] for i in range(len(zernike_moments))}

In [7]:
def compute_lbp_feature(image): # 10 features
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image, R=1, P=8, method="nri_uniform")
    hist, bins = np.histogram(lbp.flatten(), bins=10, range=(0, 10))
    return {f"LBP_{i}": hist[i] for i in range(len(hist))}

In [8]:
def compute_texture_feature(image): # 4 features
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, bins = np.histogram(image.flatten(), bins=256, range=(0, 256), density=True)
    bins = bins[:-1]
    mean = np.sum(hist * bins)
    std = np.sqrt(np.sum((bins - mean)**2 * hist))
    uniformity = np.sum(hist**2)
    third_moment = np.sum((bins - mean)**3 * hist)
    return {
        "texture_mean": mean,
        "texture_std": std,
        "texture_uniformity": uniformity,
        "texture_third_moment": third_moment
    }

In [9]:
def compute_color_feature(image):


    # BGR
    B, G, R = cv2.split(image)
    mean_R, mean_G, mean_B = np.mean(R), np.mean(G), np.mean(B)
    sqrt_R, sqrt_G, sqrt_B = np.sqrt(mean_R), np.sqrt(mean_G), np.sqrt(mean_B)
    std_R, std_G, std_B = np.std(R), np.std(G), np.std(B)
    skew_R, skew_G, skew_B = skew(R.flatten()), skew(G.flatten()), skew(B.flatten())
    kur_R, kur_G, kur_B = kurtosis(R.flatten()), kurtosis(G.flatten()), kurtosis(B.flatten())


    # HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    mean_h, mean_s, mean_v = np.mean(h), np.mean(s), np.mean(v)
    std_h, std_s, std_v = np.std(h), np.std(s), np.std(v)
    sqrt_h, sqrt_s, sqrt_v = np.sqrt(mean_h), np.sqrt(mean_s), np.sqrt(mean_v)
    skew_h, skew_s, skew_v = skew(h.flatten()), skew(s.flatten()), skew(v.flatten())
    kur_h, kur_s, kur_v = kurtosis(h.flatten()), kurtosis(s.flatten()), kurtosis(v.flatten())


    # Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    mean_l, mean_a, mean_b = np.mean(l), np.mean(a), np.mean(b)
    std_l, std_a, std_b = np.std(l), np.std(a), np.std(b)
    sqrt_l, sqrt_a, sqrt_b = np.sqrt(mean_l), np.sqrt(mean_a), np.sqrt(mean_b)
    skew_l, skew_a, skew_b = skew(l.flatten()), skew(a.flatten()), skew(b.flatten())
    kur_l, kur_a, kur_b = kurtosis(l.flatten()), kurtosis(a.flatten()), kurtosis(b.flatten())


    # YCbCr
    ycrcb_img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb_img)
    mean_y, mean_cr, mean_cb = np.mean(y), np.mean(cr), np.mean(cb)
    std_y, std_cr, std_cb = np.std(y), np.std(cr), np.std(cb)
    sqrt_y, sqrt_cr, sqrt_cb = np.sqrt(mean_y), np.sqrt(mean_cr), np.sqrt(mean_cb)
    skew_y, skew_cr, skew_cb = skew(y.flatten()), skew(cr.flatten()), skew(cb.flatten())
    kur_y, kur_cr, kur_cb = kurtosis(y.flatten()), kurtosis(cr.flatten()), kurtosis(cb.flatten())


    # XYZ
    xyz_img = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    X, Y, Z = cv2.split(xyz_img)
    mean_X, mean_Y, mean_Z = np.mean(X), np.mean(Y), np.mean(Z)
    std_X, std_Y, std_Z = np.std(X), np.std(Y), np.std(Z)
    sqrt_X, sqrt_Y, sqrt_Z = np.sqrt(mean_X), np.sqrt(mean_Y), np.sqrt(mean_Z)
    skew_X, skew_Y, skew_Z = skew(X.flatten()), skew(Y.flatten()), skew(Z.flatten())
    kur_X, kur_Y, kur_Z = kurtosis(X.flatten()), kurtosis(Y.flatten()), kurtosis(Z.flatten())


    return {"mean_r": mean_R, "mean_g": mean_G, "mean_B": mean_B,
            "sqrt_r": sqrt_R, "sqrt_g": sqrt_G, "sqrt_B": sqrt_B,
            "std_r": std_R, "std_g": std_G, "std_B": std_B,
            "skew_r": skew_R, "skew_g": skew_G, "skew_B": skew_B,
            "kur_r": kur_R, "kur_g": kur_G, "kur_B": kur_B,


            "mean_h": mean_h, "mean_s": mean_s, "mean_v": mean_v,
            "std_h": std_h, "std_s": std_s, "std_v": std_v,
            "sqrt_h": sqrt_h, "sqrt_s": sqrt_s, "sqrt_v": sqrt_v,
            "skew_h": skew_h, "skew_s": skew_s, "skew_v": skew_v,
            "kur_h": kur_h, "kur_s": kur_s, "kur_v": kur_v,
    

            "mean_l": mean_l, "mean_a": mean_a, "mean_b": mean_b,
            "std_l": std_l, "std_a": std_a, "std_b": std_b,
            "sqrt_l": sqrt_l, "sqrt_a": sqrt_a, "sqrt_b": sqrt_b,
            "skew_l": skew_l, "skew_a": skew_a, "skew_b": skew_b,
            "kur_l": kur_l, "kur_a": kur_a, "kur_b": kur_b,


            "mean_y": mean_y, "mean_cb": mean_cb, "mean_cr": mean_cr,
            "std_y": std_y, "std_cb": std_cb, "std_cr": std_cr,
            "sqrt_y": sqrt_y, "sqrt_cb": sqrt_cb, "sqrt_cr": sqrt_cr,
            "skew_y": skew_y, "skew_cb": skew_cb, "skew_cr": skew_cr,
            "kur_y": kur_y, "kur_cb": kur_cb, "kur_cr": kur_cr,

            "mean_x": mean_X, "mean_Y": mean_Y, "mean_z": mean_Z,
            "std_x": std_X, "std_Y": std_Y, "std_z": std_Z,
            "sqrt_x": sqrt_X, "sqrt_Y": sqrt_Y, "sqrt_z": sqrt_Z,
            "skew_x": skew_X, "skew_Y": skew_Y, "skew_z": skew_Z,
            "kur_x": kur_X, "kur_Y": kur_Y, "kur_z": kur_Z,

            }

In [ ]:
def compute_basic_feature(image):  # 8 features
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return {
            "area": 0, "length": 0, "width": 0, "ratio": 0,
            "major_axis_length": 0, "minor_axis_length": 0,
            "convex_hull_area": 0, "convex_hull_perimeter": 0,
            "shape_factor_1": 0, "shape_factor_2": 0,
            "shape_factor_3": 0, "shape_factor_4": 0,
            "equivalent_diameter": 0, "aspect_ratio": 0,
            "perimeter": 0, "roundness": 0,
            "compactness": 0, "solidity": 0
        }
    contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(contour)

    peri = cv2.arcLength(contour, True)
    x, y, w, h = cv2.boundingRect(contour)

    length = x + w
    width = y + h
    ratio = length / width if width != 0 else 0
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        major_axis = max(ellipse[1])
        minor_axis = min(ellipse[1])
    else:
        major_axis = 0
        minor_axis = 0
    # ellipse = cv2.fitEllipse(contour)
    # major_axis = max(ellipse[1])
    # minor_axis = min(ellipse[1])

    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)

    sf1 = major_axis / area if area != 0 else 0
    sf2 = minor_axis / area if area != 0 else 0
    sf3 = area / ((0.5 * major_axis)**2 * np.pi) if major_axis != 0 else 0
    sf4 = area / (0.5**2 * major_axis * minor_axis * np.pi) if major_axis * minor_axis != 0 else 0

    ed = np.sqrt(4 * area / np.pi) if area != 0 else 0
    ar = major_axis / minor_axis if minor_axis != 0 else 0
    roundness = (4 * area * np.pi) / peri**2 if peri != 0 else 0
    Co = ed / major_axis if major_axis != 0 else 0
    solid = area / hull_area if hull_area != 0 else 0

    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis,
        "minor_axis_length": minor_axis,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
        "shape_factor_1": sf1,
        "shape_factor_2": sf2,
        "shape_factor_3": sf3,
        "shape_factor_4": sf4,
        "equivalent_diameter": ed,
        "aspect_ratio": ar,
        "perimeter": peri,
        "roundness": roundness,
        "compactness": Co,
        "solidity": solid
    }


In [ ]:
def compute_glcm_descriptor(image): # 16 features
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    distance = [3]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']

    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)

    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)

    return {f"GLCM_{i}": features[i] for i in range(len(features))}

In [12]:
def compute_percentile_features(image):
    features = {}

    percentiles = [5, 25, 50, 75, 95]

    for color_space, prefix, channel_names in [
        (None, 'bgr', ['B', 'G', 'R']),
        (cv2.COLOR_BGR2HSV, 'hsv', ['H', 'S', 'V']),
        (cv2.COLOR_BGR2LAB, 'lab', ['L', 'A', 'B']),
        (cv2.COLOR_BGR2YCrCb, 'ycrcb', ['Y', 'Cr', 'Cb']),
        (cv2.COLOR_BGR2XYZ, 'xyz', ['X', 'Y', 'Z'])
    ]:
        if color_space is None:
            converted = image
        else:
            converted = cv2.cvtColor(image, color_space)

        channels = cv2.split(converted)

        for i, channel_name in enumerate(channel_names):
            for p in percentiles:
                value = np.percentile(channels[i], p)
                features[f'pf_{prefix}_{channel_name.lower()}_p{p}'] = value

    return features


In [13]:
def compute_color_variance_ratios(image):
    features = {}

    # BGR
    B, G, R = cv2.split(image)
    var_B, var_G, var_R = np.var(B), np.var(G), np.var(R)

    # Variance ratios
    features['var_ratio_R_G'] = var_R / (var_G + 1e-7)
    features['var_ratio_R_B'] = var_R / (var_B + 1e-7)
    features['var_ratio_G_B'] = var_G / (var_B + 1e-7)

    # HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv_img)
    var_H, var_S, var_V = np.var(H), np.var(S), np.var(V)

    features['var_ratio_H_S'] = var_H / (var_S + 1e-7)
    features['var_ratio_H_V'] = var_H / (var_V + 1e-7)
    features['var_ratio_S_V'] = var_S / (var_V + 1e-7)

    # Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L, A, B_lab = cv2.split(lab_img)
    var_L, var_A, var_B_lab = np.var(L), np.var(A), np.var(B_lab)

    features['var_ratio_L_A'] = var_L / (var_A + 1e-7)
    features['var_ratio_L_B'] = var_L / (var_B_lab + 1e-7)
    features['var_ratio_A_B'] = var_A / (var_B_lab + 1e-7)

    return features

In [14]:
def compute_color_range_features(image):
    features = {}
    iters = [(None, 'bgr', ['B', 'G', 'R']),
             (cv2.COLOR_BGR2HSV, 'hsv', ['H', 'S', 'V']),
             (cv2.COLOR_BGR2LAB, 'lab', ['L', 'A', 'B']),
             (cv2.COLOR_BGR2YCrCb, 'ycrcb', ['Y', 'Cr', 'Cb']),
             (cv2.COLOR_BGR2XYZ, 'xyz', ['X', 'Y', 'Z'])
    ]
    for color_space, prefix, channel_names in iters:
        if color_space is None:
            converted = image
        else:
            converted = cv2.cvtColor(image, color_space)

        channels = cv2.split(converted)

        for i, channel_name in enumerate(channel_names):
            channel = channels[i]

            # Range
            features[f'range_{prefix}_{channel_name.lower()}_range'] = np.max(channel) - np.min(channel)

            # Interquartile range
            features[f'iqr_{prefix}_{channel_name.lower()}_iqr'] = np.percentile(channel, 75) - np.percentile(channel, 25)

            # Mode + mode concentration
            hist, bin_edges = np.histogram(channel, bins=256, range=(0, 256))
            mode_bin = np.argmax(hist)
            mode_value = (bin_edges[mode_bin] + bin_edges[mode_bin + 1]) / 2
            mode_concentration = hist[mode_bin] / np.sum(hist)

            features[f'mv_{prefix}_{channel_name.lower()}_mode'] = mode_value
            features[f'mc_{prefix}_{channel_name.lower()}_mode_conc'] = mode_concentration

    return features

## SIFT + GIST laau qua nen chua thu

In [ ]:
import cv2
import numpy as np
from skimage import filters, transform
from scipy import fftpack

def compute_sift_features(image):

    if image is None:
        return {}

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()

    keypoints, descriptors = sift.detectAndCompute(gray, None)

    features = {}
    
    if descriptors is not None and len(descriptors) > 0:
        # Summarize descriptors
        features['sift_descriptor_mean'] = np.mean(descriptors, axis=0).mean()  
        features['sift_descriptor_std'] = np.std(descriptors, axis=0).mean()   
        features['sift_keypoint_count'] = len(keypoints)                    
    else:
        features['sift_descriptor_mean'] = 0.0
        features['sift_descriptor_std'] = 0.0
        features['sift_keypoint_count'] = 0

    return features

def compute_gist_features(image, nblocks=4, orientations_per_scale=(8, 8, 4)):

    if image is None:
        return {}

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = transform.resize(gray, (128, 128), anti_aliasing=True)

    # Parameters
    nblocks = nblocks  
    block_size = 128 // nblocks  # Size block
    features = []

    scales = len(orientations_per_scale)
    frequencies = [0.2, 0.1, 0.05][:scales]  
    orientations = orientations_per_scale

    # Apply Gabor filters
    for scale, (freq, n_orient) in enumerate(zip(frequencies, orientations)):
        for theta in np.linspace(0, np.pi, n_orient, endpoint=False):
            #Gabor filter
            sigma = 1.0 / freq
            kernel = cv2.getGaborKernel((21, 21), sigma, theta, 1.0 / freq, 0.5, 0, ktype=cv2.CV_32F)
            
            # Filter image
            filtered = cv2.filter2D(gray, cv2.CV_32F, kernel)
            
            for i in range(nblocks):
                for j in range(nblocks):
                    block = filtered[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
                    energy = np.mean(block**2)
                    features.append(energy)

    feature_dict = {f'gist_{i}': val for i, val in enumerate(features)}
    
    return feature_dict

In [ ]:
def extract_all_features(image):

    features = {}

    lbp_hist = compute_lbp_feature(image)
    features.update(lbp_hist)

    texture_features = compute_texture_feature(image)
    features.update(texture_features)

    zernike_features = compute_zernike_moments(image, degree=8)
    features.update(zernike_features)

    basic_features = compute_basic_feature(image)
    features.update(basic_features)

    color_features = compute_color_feature(image)
    features.update(color_features)

    glcm_features = compute_glcm_descriptor(image)
    features.update(glcm_features)

    percentile_features = compute_percentile_features(image)
    features.update(percentile_features)

    color_variance_ratios = compute_color_variance_ratios(image)
    features.update(color_variance_ratios)

    color_range_features = compute_color_range_features(image)
    features.update(color_range_features)

    # sift_features = compute_sift_features(image)
    # features.update(sift_features)

    # gist_features = compute_gist_features(image)
    # features.update(gist_features)

    return features

In [17]:
def process_directory(base_path):
    all_data = []
    image_paths = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = 0 if 'negative' in root else 1
                image_paths.append((os.path.join(root, file), label))

    for image_path, label in tqdm(image_paths, desc="Processing Images"):
        image = cv2.imread(image_path)
        if image is None:
            continue

        features = extract_all_features(image)
        features["Label"] = label

        all_data.append(features)

    data = pd.DataFrame(all_data)

    return data

In [ ]:
def process_dataset(image_dir):
    for subdir in tqdm(os.listdir(image_dir), desc="Processing subdirectories"):
        subdir_path = os.path.join(image_dir, subdir)
        if os.path.isdir(subdir_path):
            label = subdir 
            dataset_rows = []
            for filename in os.listdir(subdir_path):
                if filename.endswith(('.jpg', '.png', '.jpeg')):
                    img_path = os.path.join(subdir_path, filename)
                    image = cv2.imread(img_path)
                    if image is not None:
                        features = extract_all_features(image)
                        row = {'label': label}
                        row.update(features)
                        dataset_rows.append(row)
            if dataset_rows:
                df = pd.DataFrame(dataset_rows)
                output_path = f'leaf_features_{label}.csv'
                df.to_csv(output_path,index=False)
    return None


In [ ]:
image_dir = "/kaggle/input/segmented-images-leaves"
process_dataset(image_dir)
print("All subdirectories processed and saved as separate CSV files.")

Processing subdirectories:  28%|██▊       | 5/18 [37:29<1:22:25, 380.43s/it] 